# 1. はじめに

回帰モデルの作成において、モデルのブレンディングという手法があリます。 <br>
データを加工した後XGBR, LGBMなどで複数のベースモデルを作成し、算出されたスコアをさらに特徴量とみなしてメタモデルを作成するというものです。 <br>
一見シンプルなアルゴリズムですが、ハイスコアを叩き出しているNotebookが散見されるので勉強も兼ねて試してみることにします。

## データ概要

本コンペはHomeCredit社における借り入れの不履行を予測するものになってます。
顧客の年齢や性別、過去の他社借入の状況を特徴量としてモデルを作成します。

データ構造を軽く解説します。

◯application_train, application_test<br>
HomeCredit社の借入申し込み情報です。性別や年齢、職業、学歴などの基本的な属性が入ってます。<br>
当然ですがtestデータにはデフォルトの是非（二値データ）が入っていません。

◯bureau<br>
他社の借り入れ情報です。<br>
申し込みデータとSK＿ID＿CURRというID変数で結び付きます。同一のSK_ID_CURRに対して複数の借入データが存在する場合もあります。
   
◯previous_application<br>
HomeCredit社における過去の申し込み情報です。<br>
過去のIDと現在のID、そして申し込み額や最終的な借入額などが入っています。申込データとSK_ID_CURRで結び付きます。
 
他にも色々ありますが、今回の分析ではとりあえず以上の3つを用います。

![](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

In [ ]:
# ライブラリの読み込み

import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import seaborn as sns

# データの読み込み

train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
br = pd.read_csv('/kaggle/input/home-credit-default-risk/bureau.csv')
pr = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')

# 2.EDA

## 2.1 申込データについて

Applicationデータについてデータ処理およびデフォルト率の調査を行います。

In [ ]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

train.head()

各変数ごとに欠損値の数を表示します。

In [ ]:
# 変数ごとに欠損値の数を表示します

total = train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False)
missing_application_train = pd.concat([total, percent], axis = 1, keys = ['total', 'percent'])
missing_application_train.head(68)

欠損値が無くても、明らかにおかしな値が記録されているデータもあるので注意が必要です。

### 2.1.1 Label Encoding

性別や各フラグ変数を二値データに変換します。

In [ ]:
le = LabelEncoder()
le_count = 0

for col in train:
    if train[col].dtype == 'object':
        if len(list(train[col].unique())) <= 2:
            le.fit(train[col])
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])
        
            le_count += 1

print('%d columns were label encoded.' % le_count) #　二値データに変換された変数の数を出力

In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

print('Training Features shape:', train.shape)
print('Testing Features shape:', test.shape)

テストデータのコラム数は訓練データのそれより1つ多い（ターゲット変数）状態になっていないといけないので、両方に共通している特徴量のみ残します。

In [ ]:
# alignメソッドでコラム数を調整

train_target = train['TARGET']
train, test = train.align(test, join = 'inner', axis = 1)

train['TARGET'] = train_target

print('Training Features shape:', train.shape)
print('Testing Features shape:', test.shape)
train.head()

### 2.1.2 相関係数

各変数のターゲットとのPearson相関係数を調べます。

In [ ]:
correlations = train.corr()['TARGET'].sort_values()

print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

・正の相関・・・REGION_RATING（住む地域に対する信用評価）が大きな相関を持っていることがわかります。

・負の相関・・・EXT_SOURCE（外部スコア）が大きい相関を持つことがわかります。

### 2.1.3 基本属性

年齢、性別、年収、貸与額、子供の数、独既区分、勤続日数（それぞれDAYS_BIRTH, CODE_GENDER, AMT_INCOME_TOTAL, AMT_CREDIT, CNT_CHILDREN, CNT_FAM_MEMBERS, DAYS_EMPLOYEDが対応）等の基本属性について調べていきます。

#### 年齢
<br>
年齢は生まれてからの日数に負号をつけたもので表されていますので、見やすい数字に変換します。<br>
訓練データとテストデータの両方に変換を行う必要があるところに注意です。

In [ ]:
# -365で割ることで、小数点付きの年齢に変換します

train['AGE'] = train['DAYS_BIRTH']/-365
train.drop(columns = ['DAYS_BIRTH'], inplace = True)
test['AGE'] = test['DAYS_BIRTH']/-365
test.drop(columns = ['DAYS_BIRTH'], inplace = True)

訓練データに対しKDEプロットを作成します。

In [ ]:
plt.figure(figsize = (10, 8))

sns.kdeplot(train.loc[train['TARGET'] == 0, 'AGE'] / 365, label = 'TARGET == 0')
sns.kdeplot(train.loc[train['TARGET'] == 1, 'AGE'] / 365, label = 'TARGET == 1')
plt.legend(labels = ['TARGET == 0', "TARGET == 1"])

plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distibution of Ages')

若い人ほどデフォルト率が高いように思えます。<br>
年齢変数をビン化し、各ビンにおけるデフォルト率を算出します。

In [ ]:
#Age information into a separate detaframe
age_data = train[['TARGET', 'AGE']]
age_data['AGE_BINNED'] = pd.cut(age_data['AGE'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
age_groups = age_data.groupby('AGE_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to repay(%)')
plt.title('Failure to repay by Age Group')

年齢の増加に伴い、右肩下がりでデフォルト率が下がっていることがわかります。<br>
モデル構築の有用な特徴量になりそうです。

### 性別

性別ごとにデフォルト率を算出します。

In [ ]:
print('男性のデフォルト率 : ' + str(train[train['CODE_GENDER_M'] == 1]['TARGET'].sum() / len(train)))
print('女性のデフォルト率 : ' + str(train[train['CODE_GENDER_M'] == 0]['TARGET'].sum() / len(train)))

女性の方が比較的デフォルトしやすいように見えますがが, 1%程度の差しかなく有意な差とは言い難いかもしれません。

### 年収

年収(AMT_INCOME_TOTAL)について平均、最大などの統計値を求めます。

In [ ]:
train['AMT_INCOME_TOTAL'].describe()

単位はドルですが、最大値が異常に大きいように思えます（年収１億ドル以上？）<br>
外れ値として除外することもできますが、今回は年齢と同様にビン化処理します。<br>
分割の仕方は0から10万,10から３０万,30から５０万,50から100万,それ以上という形である程度恣意的に決定します。

In [ ]:
#Income information into a separate detaframe
income_data = train[['SK_ID_CURR', 'TARGET', 'AMT_INCOME_TOTAL']]
income_data['INCOME_BINNED'] = pd.cut(income_data['AMT_INCOME_TOTAL'], [0, 1e+05, 3e+05, 5e+05, 1e+06, 2e+08])
income_data.head(10)

In [ ]:
income_groups = income_data.groupby('INCOME_BINNED').mean()
income_groups

In [ ]:
plt.figure(figsize = (8, 8))

plt.bar(income_groups.index.astype(str), 100 * income_groups['TARGET'])

plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to repay(%)')
plt.title('Failure to repay by Total Income Group')

年収が低いほどデフォルトに至りやすいようです。直感には合います。

### 貸与額

まずは統計量を算出します。

In [ ]:
train['AMT_CREDIT'].describe()

五等分にビン化します。

In [ ]:
#Income information into a separate detaframe
credit_data = train[['SK_ID_CURR', 'TARGET', 'AMT_CREDIT']]
credit_data['CREDIT_BINNED'] = pd.cut(credit_data['AMT_CREDIT'], 5)
credit_data.head(10)

In [ ]:
credit_groups = credit_data.groupby('CREDIT_BINNED').mean()
credit_groups

In [ ]:
plt.figure(figsize = (8, 8))

plt.bar(credit_groups.index.astype(str), 100 * credit_groups['TARGET'])

plt.xticks(rotation = 75); plt.xlabel('Amt_Credit Group (Doller)'); plt.ylabel('Failure to repay(%)')
plt.title('Failure to repay by Credit Group')

低い貸付額の顧客程、直感に反して不履行に至りやすいように見えます。 <br>
貸付額が高い顧客も比較的デフォルトしやすいですが、これは宜なるかなという感じですね。

### 子供の数

貸付額と同様ビン化します。

In [ ]:
train['CNT_CHILDREN'].describe()

In [ ]:
#Income information into a separate detaframe
child_data = train[['TARGET', 'CNT_CHILDREN']]
child_data['CHILDREN_BINNED'] = pd.cut(child_data['CNT_CHILDREN'], 5)

child_groups = child_data.groupby('CHILDREN_BINNED').mean()
child_groups

In [ ]:
plt.figure(figsize = (8, 8))

plt.bar(child_groups.index.astype(str), 100 * child_groups['TARGET'])

plt.xticks(rotation = 75); plt.xlabel('Child Group (Doller)'); plt.ylabel('Failure to repay(%)')
plt.title('Failure to repay by Children number')

子供が多いほどデフォルトしやすいようです。 <br>
地域によって家庭の子供の数は違いがあると考えられ、顧客の居住エリアとも相関がありそうです。

### 勤続日数

In [ ]:
train['DAYS_EMPLOYED_1'] = abs(train['DAYS_EMPLOYED'])
train.drop(columns = ['DAYS_EMPLOYED'], inplace = True)
train['DAYS_EMPLOYED_1'].describe()

最大値が明らかに大きく（100年以上働いてる？）、外れ値と見做して処理します。

In [ ]:
print(len(train[train['DAYS_EMPLOYED_1'] == 365243]))

55,374件のデータの勤続日数が同じ外れ値を共有しています。<br>
こちらの（https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction） 素晴らしいNotebookによると、これらの顧客はデフォルト率が低いことがわかっています。<br>
したがって単に消去せず、ビン化して処理していきます。

In [ ]:
#Income information into a separate detaframe
employ_data = train[['SK_ID_CURR', 'TARGET', 'DAYS_EMPLOYED_1']]
employ_data['DAYS_EMPLOYED_BINNED'] = pd.cut(employ_data['DAYS_EMPLOYED_1'], [0, 933, 2218, 5707, 365243])

employ_groups = employ_data.groupby('DAYS_EMPLOYED_BINNED').mean()
employ_groups

In [ ]:
plt.figure(figsize = (8, 8))

plt.bar(employ_groups.index.astype(str), 100 * employ_groups['TARGET'])

plt.xticks(rotation = 75); plt.xlabel('Employ Days Group (Days)'); plt.ylabel('Failure to repay(%)')
plt.title('Failure to repay by Employed days')

勤続日数が長いほど事故しにくいようです。

以上より相関の有りそうな変数と、事故変数との相関係数の絶対値が大きいものを残します。

In [ ]:
# 変数選択　年収、勤続日数はビン化

train_1 = train[['SK_ID_CURR', 'TARGET', 'AGE', 'CODE_GENDER_M', 'AMT_CREDIT', 'CNT_CHILDREN', 'REGION_RATING_CLIENT_W_CITY', 
               'NAME_INCOME_TYPE_Working', 'REGION_POPULATION_RELATIVE', 'FLOORSMAX_AVG', 'NAME_INCOME_TYPE_Pensioner',
               'NAME_EDUCATION_TYPE_Higher education']]

income_bin = income_data[['SK_ID_CURR', 'INCOME_BINNED']]
employ_bin = employ_data[['SK_ID_CURR', 'DAYS_EMPLOYED_BINNED']]
train_2 = pd.merge(train_1, income_bin, on = 'SK_ID_CURR')
train_3 = pd.merge(train_2, employ_bin, on = 'SK_ID_CURR')

train_3['INCOME_BIN'] = 1
train_3['DAYS_EMPLOYED_BIN'] = 1

#ビン化処理
for i in range(len(train_3)):
    if train_3['INCOME_BINNED'][i] == '(0.0, 10000.0]':
        train_3['INCOME_BIN'][i] = 1
    elif train_3['INCOME_BINNED'][i] == '(10000.0, 30000.0]':
        train_3['INCOME_BIN'][i] = 2
    elif train_3['INCOME_BINNED'][i] == '(30000.0, 50000.0]':
        train_3['INCOME_BIN'][i] = 3
    elif train_3['INCOME_BINNED'][i] == '(50000.0, 100000.0]':
        train_3['INCOME_BIN'] = 4
    else:
        train_3['INCOME_BIN'][i] = 5
    
for i in range(len(train_3)):
    if train_3['DAYS_EMPLOYED_BINNED'][i] == '(0, 933]':
        train_3['DAYS_EMPLOYED_BIN'][i] = 1
    elif train_3['DAYS_EMPLOYED_BINNED'][i] == '(933, 2218]':
        train_3['DAYS_EMPLOYED_BIN'][i] = 2
    elif train_3['DAYS_EMPLOYED_BINNED'][i] == '(2218, 5707]':
        train_3['DAYS_EMPLOYED_BIN'][i] = 3
    else:
        train_3['DAYS_EMPLOYED_BIN'][i] = 4

train_3.drop(columns = ['DAYS_EMPLOYED_BINNED', 'INCOME_BINNED'], inplace = True)

In [ ]:
# テストデータにも同様の処理

test_1 = test[['SK_ID_CURR', 'AGE', 'CODE_GENDER_M', 'AMT_CREDIT', 'CNT_CHILDREN', 'REGION_RATING_CLIENT_W_CITY', 
               'NAME_INCOME_TYPE_Working', 'REGION_POPULATION_RELATIVE', 'FLOORSMAX_AVG', 'NAME_INCOME_TYPE_Pensioner',
               'NAME_EDUCATION_TYPE_Higher education']]

#Income information into a separate detaframe
income_data_test = test[['SK_ID_CURR', 'AMT_INCOME_TOTAL']]
income_data_test['INCOME_BINNED'] = pd.cut(income_data_test['AMT_INCOME_TOTAL'], [0, 1e+05, 3e+05, 5e+05, 1e+06, 2e+08])
test['DAYS_EMPLOYED_1'] = abs(test['DAYS_EMPLOYED'])
employ_data_test = test[['SK_ID_CURR', 'DAYS_EMPLOYED_1']]
employ_data_test['DAYS_EMPLOYED_BINNED'] = pd.cut(employ_data_test['DAYS_EMPLOYED_1'], [0, 933, 2218, 5707, 365243])


income_bin_test = income_data_test[['SK_ID_CURR', 'INCOME_BINNED']]
employ_bin_test = employ_data_test[['SK_ID_CURR', 'DAYS_EMPLOYED_BINNED']]
test_2 = pd.merge(test_1, income_bin_test, on = 'SK_ID_CURR')
test_3 = pd.merge(test_2, employ_bin_test, on = 'SK_ID_CURR')

test_3['INCOME_BIN'] = 1
test_3['DAYS_EMPLOYED_BIN'] = 1

#ビン化処理
for i in range(len(test_3)):
    if test_3['INCOME_BINNED'][i] == '(0.0, 10000.0]':
        test_3['INCOME_BIN'][i] = 1
    elif test_3['INCOME_BINNED'][i] == '(10000.0, 30000.0]':
        test_3['INCOME_BIN'][i] = 2
    elif test_3['INCOME_BINNED'][i] == '(30000.0, 50000.0]':
        test_3['INCOME_BIN'][i] = 3
    elif test_3['INCOME_BINNED'][i] == '(50000.0, 100000.0]':
        test_3['INCOME_BIN'] = 4
    else:
        test_3['INCOME_BIN'][i] = 5
    
for i in range(len(test_3)):
    if test_3['DAYS_EMPLOYED_BINNED'][i] == '(0, 933]':
        test_3['DAYS_EMPLOYED_BIN'][i] = 1
    elif test_3['DAYS_EMPLOYED_BINNED'][i] == '(933, 2218]':
        test_3['DAYS_EMPLOYED_BIN'][i] = 2
    elif test_3['DAYS_EMPLOYED_BINNED'][i] == '(2218, 5707]':
        test_3['DAYS_EMPLOYED_BIN'][i] = 3
    else:
        test_3['DAYS_EMPLOYED_BIN'][i] = 4

test_3.drop(columns = ['DAYS_EMPLOYED_BINNED', 'INCOME_BINNED'], inplace = True)

## 2.2 bureauデータ

いわゆるCICデータのようなものです。こちらについて調べていきます。

In [ ]:
br

・DAYS_CREDIT：申し込み時点での、各借り入れの申し込み日<br>
・CREDIT_DAY_OVERDUE：返済日から経過した日数<br>
・DAYS_CREDIT_ENDDATE：返済日までの残り日数<br>
この辺はIDごとの総和とかが効いてきそうですが、とりあえず今回は件数のみ算出します。

In [ ]:
# SK_ID_CURRについてgroupbyメソッドで件数を数え、列名を変えます。
previous_loan_counts = br.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

# 訓練データに結合
train_3 = train_3.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

# 欠損値であるということは過去に他社借入がないということなので、０で置き換えます。
train_3['previous_loan_counts'] = train_3['previous_loan_counts'].fillna(0)
train_3.head()

In [ ]:
#テストデータにも同様の処理を行います。

previous_loan_counts = br.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

test_3 = test_3.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

test_3['previous_loan_counts'] = test_3['previous_loan_counts'].fillna(0)
test_3.head()

## 2.3 previous loan application

HomeCredit社における過去の申し込み情報です。

In [ ]:
pr

候補の変数として<br>
・件数カウント<br>
・最終的な借入額 (AMT_CREDIT) の平均、最大、最小<br>
などが考えられます。

In [ ]:
#件数カウント

previous_application_counts = pr.groupby('SK_ID_CURR', as_index=False)['SK_ID_PREV'].count().rename(columns = {'SK_ID_PREV': 'previous_application_counts'})
previous_application_counts

train_3 = train_3.merge(previous_application_counts, on = 'SK_ID_CURR', how = 'left')

# 欠損値は０で置き換えます。
train_3['previous_application_counts'] = train_3['previous_application_counts'].fillna(0)
train_3.head()

In [ ]:
#平均、最大、最小を算出します。

pa_mean = pr.groupby('SK_ID_CURR', as_index = False).mean()[['SK_ID_CURR', 'AMT_CREDIT']].rename(columns = {'AMT_CREDIT' : 'AMT_CREDIT_MEAN'})
pa_max = pr.groupby('SK_ID_CURR', as_index = False).max()[['SK_ID_CURR', 'AMT_CREDIT']].rename(columns = {'AMT_CREDIT' : 'AMT_CREDIT_MAX'})
pa_min = pr.groupby('SK_ID_CURR', as_index = False).min()[['SK_ID_CURR', 'AMT_CREDIT']].rename(columns = {'AMT_CREDIT' : 'AMT_CREDIT_MIN'})

train_3 = train_3.merge(pa_mean, on = 'SK_ID_CURR', how = 'left')
train_3 = train_3.merge(pa_max, on = 'SK_ID_CURR', how = 'left')
train_3 = train_3.merge(pa_min, on = 'SK_ID_CURR', how = 'left')

train_3

In [ ]:
#テストデータにも同様の処理を行います

previous_application_counts = pr.groupby('SK_ID_CURR', as_index=False)['SK_ID_PREV'].count().rename(columns = {'SK_ID_PREV': 'previous_application_counts'})
previous_application_counts

test_3 = test_3.merge(previous_application_counts, on = 'SK_ID_CURR', how = 'left')

# Fill the missing values with 0 
test_3['previous_application_counts'] = test_3['previous_application_counts'].fillna(0)

pa_mean = pr.groupby('SK_ID_CURR', as_index = False).mean()[['SK_ID_CURR', 'AMT_CREDIT']].rename(columns = {'AMT_CREDIT' : 'AMT_CREDIT_MEAN'})
pa_max = pr.groupby('SK_ID_CURR', as_index = False).max()[['SK_ID_CURR', 'AMT_CREDIT']].rename(columns = {'AMT_CREDIT' : 'AMT_CREDIT_MAX'})
pa_min = pr.groupby('SK_ID_CURR', as_index = False).min()[['SK_ID_CURR', 'AMT_CREDIT']].rename(columns = {'AMT_CREDIT' : 'AMT_CREDIT_MIN'})

test_3 = test_3.merge(pa_mean, on = 'SK_ID_CURR', how = 'left')
test_3 = test_3.merge(pa_max, on = 'SK_ID_CURR', how = 'left')
test_3 = test_3.merge(pa_min, on = 'SK_ID_CURR', how = 'left')

test_3

# 3.変数選択

役に立ちそうな変数を複数追加してきましたが、ここから更に厳選していきます。<br>
今回はFilterMethodと呼ばれる手法で変数選択を行なっていくことにします。

## 3.1 Variance Filter

分散が小さい（≒どの顧客に対しても同じような値を取る）変数があれば、それは明らかに役には立たない特徴量でしょう。<br>
閾値を設定し、これより分散が小さい変数は消去します。今回は0.1に設定します。

In [ ]:
#分散の小さいデータを消去
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold = 0.1)
sel.fit(train_3)
print(sum(sel.get_support()))

train_3.head(5)

## 3.2 相関係数

相関が高い二つの特徴量は、多重共線性を防ぐため一方を消去すべきです。<br>
今回は互いの相関係数が0.8以上の変数の組があれば、片方は除去することにします。

In [ ]:
#Pearson相関係数で評価
threshold = 0.8

feat_corr = set()
corr_matrix = train_3.corr()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            feat_name = corr_matrix.columns[i]
            feat_corr.add(feat_name)

print(len(set(feat_corr)))

train_3.drop(labels = feat_corr, axis = 1, inplace = True)
print(len(train_3.columns))

以上でデータ整形が終わりました。Part2にてBaseModel, MetaModelの作成をしていきます。

In [ ]:
train_3.to_csv('Training Dataset (Feature Selected)')
test_3.to_csv('Testing Dataset (Feature Selected)')